# 3. MapReduce

For this exercise we are going to use MapReduce in local mode, i.e. we won't be running anything on the cluster!
 
## 3.1. Use the commands `head`, `cat`, `uniq`, `wc`, `sort`, `find`, `xargs`, `awk` to evaluate the NASA log file:

* Data File:  <https://github.com/scalable-infrastructure/exercise-2018/blob/master/data/nasa/NASA_access_log_Jul95.gz>
* Which page was called the most?
* What was the most frequent return code?
* How many errors occurred? What is the percentage of errors?


In [33]:
!pwd
!gzip -k -d -f  ../data/nasa/NASA_access_log_Jul95.gz

/naslx/projects/pn69si/mnmda001/mnmda003/exercise-students-2018/03_MapReduce


In [2]:
!head -n 5 ../data/nasa/NASA_access_log_Jul95

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085
burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0
199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179


In [11]:
# What was the most frequent return code?
!cat ../data/nasa/NASA_access_log_Jul95 | awk '{print $(NF-1)}'| sort | uniq -c | sort -r | head -n 1

1701534 200


In [36]:
# How many errors occurred? 
#!cat ../data/nasa/NASA_access_log_Jul95 | awk '{print $(NF-1)}'| sort | grep "[45].." | uniq -c  | sort -r
# What is the percentage of errors?
!cat ../data/nasa/NASA_access_log_Jul95 | wc -l #> ../data/temp
!cat ../data/nasa/NASA_access_log_Jul95 | awk '{print $(NF-1)}'| sort | grep "[45].." --count #>> ../data/temp
!echo "10980/1891714*100 = 0.5804260052"

1891714
10980
10980/1891714*100 = 0.5804260052


/usr/bin/sh: -c: line 0: syntax error near unexpected token `('
/usr/bin/sh: -c: line 0: `awk -v n1=(cat ../data/nasa/NASA_access_log_Jul95 | awk '{print $(NF-1)}'| sort | grep "[45].." -c) 'BEGIN {printf "%.2f", n1*0.05/100}' /dev/null'


In [ ]:
# Which page was called the most?
#!cat ../data/nasa/NASA_access_log_Jul95 | awk -F\" '{print $2}' | awk -F"[ ]+" '{print $2}' | sort | uniq -c | sort -r


## 3.2 Implement a Python version of this Unix Shell script using this script as template! Run the Python script inside an Hadoop Streaming job.

Template: <https://github.com/scalable-infrastructure/scalable-infrastructure.github.io/blob/master/src/map_reduce.py>

In [1]:
import os
os.environ["HADOOP_HOME"]="/naslx/projects/pn69si/mnmda001/software/hadoop-2.7.5"
os.environ["PATH"]="/naslx/projects/pn69si/mnmda001/software/hadoop-2.7.5/bin:"+os.environ["PATH"]

In [19]:
#!hadoop fs -put ./map_reduce-implementation.py
!rm -rf ../data/output
!hadoop jar ${HADOOP_HOME}/share/hadoop/tools/lib/hadoop-streaming-2.7.5.jar -input ../data/nasa/NASA_access_log_Jul95  -output ../data/output -file ./map_reduce-implementation.py -mapper 'map_reduce-implementation.py map' -reducer 'map_reduce-implementation.py reduce'


18/04/04 15:01:28 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [./map_reduce-implementation.py] [] /tmp/streamjob6866329345192717869.jar tmpDir=null
18/04/04 15:01:30 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
18/04/04 15:01:30 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
18/04/04 15:01:30 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
18/04/04 15:01:32 INFO mapred.FileInputFormat: Total input paths to process : 1
18/04/04 15:01:32 INFO mapreduce.JobSubmitter: number of splits:7
18/04/04 15:01:32 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1761869414_0001
18/04/04 15:01:33 INFO mapred.LocalDistributedCacheManager: Localized file:/naslx/projects/pn69si/mnmda001/mnmda003/exercise-students-2018/03_MapReduce/map_reduce-implementation.py as file:/nas

---

## 3.3 Run the program Terasort on 1 GB of data - each record that TeraGen generates is 100 Bytes in size:

    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar teragen <number_of_records> <output_directory>

    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar terasort <input_directory> <output_directory>
    
    hadoop jar ${HADOOP_HOME}/share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.5.jar teravalidate <input_directory> <output_directory>

Measure the runtime for each step and plot the results!